In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True, reshape=False)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from t

In [0]:

batch_size = 100

In [3]:
example_X, example_ys = mnist.train.next_batch(batch_size)
example_X[0].shape

(28, 28, 1)

In [0]:
session = tf.InteractiveSession()

In [0]:
X = tf.placeholder('float', [batch_size, 28, 28, 1])
t = tf.placeholder('float', [batch_size, 10])

CONVOLUTION LAYER

In [0]:
filter_h, filter_w, filter_c, filter_n = 5, 5, 1, 30

In [0]:
W1 = tf.Variable(tf.random_normal([filter_h, filter_w, filter_c, filter_n], stddev=0.01))
b1 = tf.Variable(tf.zeros([filter_n]))

In [0]:
def convolution(X, W, b, padding, stride):

    n, h, w, c = map(lambda d: d.value, X.get_shape())
    filter_h, filter_w, filter_c, filter_n = [d.value for d in W.get_shape()]
    
    out_h = (h + 2*padding - filter_h)//stride + 1
    out_w = (w + 2*padding - filter_w)//stride + 1

    X_flat = flatten(X, filter_h, filter_w, filter_c, out_h, out_w, stride, padding)
    W_flat = tf.reshape(W, [filter_h*filter_w*filter_c, filter_n])
    
    z = tf.matmul(X_flat, W_flat) + b     # b: 1 X filter_n
    
    return tf.transpose(tf.reshape(z, [out_h, out_w, n, filter_n]), [2, 0, 1, 3])

In [0]:

def flatten(X, window_h, window_w, window_c, out_h, out_w, stride=1, padding=0):
    
    X_padded = tf.pad(X, [[0,0], [padding, padding], [padding, padding], [0,0]])

    windows = []
    for y in range(out_h):
        for x in range(out_w):
            window = tf.slice(X_padded, [0, y*stride, x*stride, 0], [-1, window_h, window_w, -1])
            windows.append(window)
    stacked = tf.stack(windows) # shape : [out_h, out_w, n, filter_h, filter_w, c]

    return tf.reshape(stacked, [-1, window_c*window_w*window_h])

In [11]:
conv_layer = convolution(X, W1, b1, padding=2, stride=1)
conv_layer

<tf.Tensor 'transpose:0' shape=(100, 28, 28, 30) dtype=float32>

ReLU

In [0]:
def relu(X):
    return tf.maximum(X, tf.zeros_like(X))

In [13]:
conv_activation_layer = relu(conv_layer)
conv_activation_layer

<tf.Tensor 'Maximum:0' shape=(100, 28, 28, 30) dtype=float32>

Max Pooling

In [0]:
def max_pool(X, pool_h, pool_w, padding, stride):    
    n, h, w, c = [d.value for d in X.get_shape()]
    
    out_h = (h + 2*padding - pool_h)//stride + 1
    out_w = (w + 2*padding - pool_w)//stride + 1

    X_flat = flatten(X, pool_h, pool_w, c, out_h, out_w, stride, padding)

    pool = tf.reduce_max(tf.reshape(X_flat, [out_h, out_w, n, pool_h*pool_w, c]), axis=3)
    return tf.transpose(pool, [2, 0, 1, 3])

In [15]:
pooling_layer = max_pool(conv_activation_layer, pool_h=2, pool_w=2, padding=0, stride=2)
pooling_layer

<tf.Tensor 'transpose_1:0' shape=(100, 14, 14, 30) dtype=float32>

Affine Layer

In [0]:
batch_size, pool_output_h, pool_output_w, filter_n = [d.value for d in pooling_layer.get_shape()]

In [0]:
hidden_size = 100

In [0]:
W2 = tf.Variable(tf.random_normal([pool_output_h*pool_output_w*filter_n, hidden_size], stddev=0.01))
b2 = tf.Variable(tf.zeros([hidden_size]))

In [0]:
def affine(X, W, b):
    n = X.get_shape()[0].value # number of samples
    X_flat = tf.reshape(X, [n, -1])
    return tf.matmul(X_flat, W) + b


In [20]:
affine_layer1 = affine(pooling_layer, W2, b2)
affine_layer1

<tf.Tensor 'add_1:0' shape=(100, 100) dtype=float32>

In [21]:

init = tf.global_variables_initializer()
init.run()
affine_layer1.eval({X:example_X, t:example_ys})[0]

array([ 0.00573266, -0.01146959,  0.0020477 ,  0.00109919, -0.00622499,
        0.00235034,  0.01003461, -0.00219972, -0.01139388,  0.00588117,
        0.00812068,  0.00526256, -0.02062925,  0.00526467, -0.01278924,
        0.00503032, -0.00163147,  0.00863405, -0.00405013,  0.01291035,
        0.01808444,  0.00633315, -0.00965838,  0.00377809, -0.00681359,
       -0.00620927, -0.00953156, -0.00087223, -0.00648868,  0.01847807,
        0.00578953, -0.00160581, -0.01713528,  0.0069445 , -0.01469908,
        0.00364548, -0.00194487, -0.00512307, -0.00450582,  0.00250203,
       -0.00811481,  0.00950175,  0.01948148,  0.0017098 , -0.00116168,
        0.00558827,  0.00165999,  0.00596239, -0.00575297, -0.0041101 ,
        0.01807101, -0.02170337,  0.00929432,  0.00289989,  0.01821265,
        0.0132655 , -0.00675214, -0.01938745, -0.00025279,  0.00216648,
       -0.00108207, -0.00476961, -0.00445348, -0.00372037, -0.00131145,
        0.01364032,  0.00605591,  0.00934236, -0.0060331 ,  0.01

In [22]:
affine_activation_layer1 = relu(affine_layer1)
affine_activation_layer1

<tf.Tensor 'Maximum_1:0' shape=(100, 100) dtype=float32>

In [23]:
affine_activation_layer1.eval({X:example_X, t:example_ys})[0]

array([0.00573266, 0.        , 0.0020477 , 0.00109919, 0.        ,
       0.00235034, 0.01003461, 0.        , 0.        , 0.00588117,
       0.00812068, 0.00526256, 0.        , 0.00526467, 0.        ,
       0.00503032, 0.        , 0.00863405, 0.        , 0.01291035,
       0.01808444, 0.00633315, 0.        , 0.00377809, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.01847807,
       0.00578953, 0.        , 0.        , 0.0069445 , 0.        ,
       0.00364548, 0.        , 0.        , 0.        , 0.00250203,
       0.        , 0.00950175, 0.01948148, 0.0017098 , 0.        ,
       0.00558827, 0.00165999, 0.00596239, 0.        , 0.        ,
       0.01807101, 0.        , 0.00929432, 0.00289989, 0.01821265,
       0.0132655 , 0.        , 0.        , 0.        , 0.00216648,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.01364032, 0.00605591, 0.00934236, 0.        , 0.01034562,
       0.        , 0.00908229, 0.001419  , 0.        , 0.00202

Affine Layer2

In [0]:
output_size = 10

In [0]:
W3 = tf.Variable(tf.random_normal([hidden_size, output_size], stddev=0.01))
b3 = tf.Variable(tf.zeros([output_size]))

In [0]:
affine_layer2 = affine(affine_activation_layer1, W3, b3)

In [0]:
init = tf.global_variables_initializer()
init.run()

In [28]:
affine_layer2.eval({X:example_X, t:example_ys})[0]

array([-5.9442432e-04,  1.3376488e-03,  1.9586542e-04, -5.2643161e-05,
        3.1402832e-04,  3.3987782e-04,  8.6094008e-04, -1.8518659e-03,
        1.7214729e-04, -3.3649011e-04], dtype=float32)

Softmax

In [0]:
def softmax(X):
    X_centered = X - tf.reduce_max(X) # to avoid overflow
    X_exp = tf.exp(X_centered)
    exp_sum = tf.reduce_sum(X_exp, axis=1)
    return tf.transpose(tf.transpose(X_exp) / exp_sum)

In [0]:
softmax_layer = softmax(affine_layer2)

In [31]:
softmax_layer.eval({X:example_X, t:example_ys})[0]

array([0.09993669, 0.10012997, 0.10001571, 0.09999085, 0.10002753,
       0.10003011, 0.10008225, 0.09981111, 0.10001334, 0.09996248],
      dtype=float32)

Cross Entropy Error

In [0]:
def cross_entropy_error(y, t):
    return -tf.reduce_mean(tf.log(tf.reduce_sum(y * t, axis=1)))

In [0]:
loss = cross_entropy_error(softmax_layer, t)

In [34]:
loss.eval({X:example_X, t:example_ys})

2.3025923

In [35]:
learning_rate = 0.1
trainer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
# number of times to iterate over training data
training_epochs = 2

In [37]:
num_batch = int(mnist.train.num_examples/batch_size)
num_batch

550

In [0]:
from tqdm import tqdm_notebook

In [39]:
for epoch in range(training_epochs):
    avg_cost = 0
    for _ in tqdm_notebook(range(num_batch)):
        train_X, train_ys = mnist.train.next_batch(batch_size)
        trainer.run(feed_dict={X:train_X, t:train_ys})
        avg_cost += loss.eval(feed_dict={X:train_X, t:train_ys}) / num_batch

    print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost), flush=True)


Epoch: 0001 cost= 0.770458344



Epoch: 0002 cost= 0.108908404


In [0]:
test_x = mnist.test.images[:batch_size]
test_t = mnist.test.labels[:batch_size]

In [0]:
def accuracy(network, t):
    
    t_predict = tf.argmax(network, axis=1)
    t_actual = tf.argmax(t, axis=1)

    return tf.reduce_mean(tf.cast(tf.equal(t_predict, t_actual), tf.float32))

In [42]:
accuracy(softmax_layer, t).eval(feed_dict={X:test_x, t:test_t})

0.98

In [0]:
session.close()